### Imports and model specifications

In [7]:
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F

plan_func = False

hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False

alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
charlie = sy.VirtualWorker(hook, id="charlie")
dan = sy.VirtualWorker(hook, id="dan")

if plan_func:
    @sy.func2plan(args_shape=[(1,)], state={"bias": th.tensor([3.0])})
    def plan_mult_3(x, state):
        bias = state.read("bias")
        x = x + bias
        return x
else:
    class Net(sy.Plan):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(1, 1)
            self.add_to_state(["fc1"])

        def forward(self, x):
            return self.fc1(x)
    
    plan_mult_3 = Net()
    plan_mult_3.build(th.tensor(1))

plan_mult_3.share(bob, charlie, crypto_provider=dan)

x = th.tensor([1.])
x_ptr = x.fix_prec().share(bob, charlie, crypto_provider=dan)

# print(x_ptr)

plan_mult_3.forward = None

# plan = plan_mult_3._send(me)
# plan.fix_prec().share(bob, charlie, crypto_provider=dan)
me._objects[x_ptr.id] = x_ptr
print(plan_mult_3.readable_plan)
plan_mult_3(x_ptr)

((26, (1, ((6, ((5, (b'torch.nn.functional.linear',)), None, (6, ((20, (25687233144, 75852968211, 'me', None, (10, ()), True)), (20, (41901982161, 41203134476, 'me', None, (10, (1, 1)), True)), (20, (62813969143, 45630771453, 'me', None, (10, (1,)), True)))), (0, ()))), (64409744639,)))),)


KeyError: 'Object "41203134476" not found on worker!!!You just tried to interact with an object ID:41203134476 on <VirtualWorker id:me #objects:39> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [2]:
# Support fetching a plan

import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F

plan_func = False

hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False

alice = sy.VirtualWorker(hook, id="alice")

if plan_func:
    @sy.func2plan(args_shape=[(1,)], state={"bias": th.tensor([3.0])})
    def plan_mult_3(x, state):
        bias = state.read("bias")
        x = x * bias
        return x
else:
    class Net(sy.Plan):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(1, 1)
            self.add_to_state(["fc1"])

        def forward(self, x):
            return self.fc1(x)
    
    plan_mult_3 = Net()
    plan_mult_3.build(th.tensor(1))

sent_plan = plan_mult_3.send(alice)

# Fetch plan
fetched_plan = alice.fetch_plan(sent_plan.id)

x = th.tensor([1.])
print(fetched_plan(x))

tensor([0.0460], requires_grad=True)


In [7]:
87328566258 in me._objects

False

In [2]:
alice._objects

{56905234459: tensor([3.]),
 8273699120: <Plan plan_mult_3 id:8273699120 owner:alice built>}

In [2]:
fetched_plan.state.

State: 

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import grid as gr

RuntimeError: 

for operator (Tensor 0) -> Tensor:
expected a value of type Tensor for argument '0' but found (Tensor, Tensor, Tensor, Tensor)
    gt_heights = reference_boxes_y2 - reference_boxes_y1
    gt_ctr_x = reference_boxes_x1 + 0.5 * gt_widths
    gt_ctr_y = reference_boxes_y1 + 0.5 * gt_heights

    targets_dx = wx * (gt_ctr_x - ex_ctr_x) / ex_widths
    targets_dy = wy * (gt_ctr_y - ex_ctr_y) / ex_heights
    targets_dw = ww * torch.log(gt_widths / ex_widths)
    targets_dh = wh * torch.log(gt_heights / ex_heights)

    targets = torch.cat((targets_dx, targets_dy, targets_dw, targets_dh), dim=1)
                         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    return targets
:
    gt_heights = reference_boxes_y2 - reference_boxes_y1
    gt_ctr_x = reference_boxes_x1 + 0.5 * gt_widths
    gt_ctr_y = reference_boxes_y1 + 0.5 * gt_heights

    targets_dx = wx * (gt_ctr_x - ex_ctr_x) / ex_widths
    targets_dy = wy * (gt_ctr_y - ex_ctr_y) / ex_heights
    targ

We also need to execute commands specific to importing/starting PySyft. We create a few workers (named `alice` and `bob`). Lastly, we define the `crypto_provider` who gives all the crypto primitives we may need ([See our tutorial on SMPC for more details](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%205%20-%20Intro%20to%20Encrypted%20Programs.ipynb)).

In [2]:
import syft as sy
hook = sy.TorchHook(torch)
me = hook.local_worker
me.is_client_worker = False

bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="bob")
bob.connect()
alice =  gr.WebsocketGridClient(hook, "http://localhost:3001", id="alice")
crypto_provider = gr.WebsocketGridClient(hook, "http://localhost:3002", id="james")
alice.connect()
crypto_provider.connect()
gr.connect_all_nodes([bob, alice, crypto_provider])

In [17]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 1)
        self.add_to_state(["fc1"])

    def forward(self, x):
        return self.fc1(x)

In [18]:
model = Net()

NameError: name 'nn' is not defined

In [5]:
data_shape = (1, 1)
data = torch.zeros(data_shape)
target = torch.tensor(1)

In [6]:
model.build(data)

In [7]:
ptr_model = model.send(bob)

In [8]:
ptr_model.id

89636588365

In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

In [1]:
import torch
import grid as gr
import syft as sy

hook = sy.TorchHook(torch)
me = hook.local_worker
me.is_client_worker = False

bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="bob")
bob.connect()
alice =  gr.WebsocketGridClient(hook, "http://localhost:3001", id="alice")
crypto_provider = gr.WebsocketGridClient(hook, "http://localhost:3002", id="james")
alice.connect()
crypto_provider.connect()
gr.connect_all_nodes([bob, alice, crypto_provider])

In [2]:
# Fetch plan
fetched_plan = bob.fetch_plan(16111384637)
print(fetched_plan.owner)
# get_plan = sent_plan.get()

# Execute it with an AST
x = torch.tensor(1)
# print(get_plan(x))
sh_x = x.share(alice, bob, crypto_provider=crypto_provider)
me._objects[sh_x.id] = sh_x
print(fetched_plan(sh_x).get())

<VirtualWorker id:me #objects:0>


Exception in thread Thread-22:
Traceback (most recent call last):
  File "/home/marianne/Grid/grid/websocket_client.py", line 68, in on_client_result
    self.response_from_client = binascii.unhexlify(args[2:-1])
binascii.Error: Odd-length string

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/marianne/anaconda3/envs/syft/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/marianne/anaconda3/envs/syft/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/marianne/anaconda3/envs/syft/lib/python3.6/site-packages/socketio/client.py", line 473, in _handle_eio_message
    self._handle_event(pkt.namespace, pkt.id, pkt.data)
  File "/home/marianne/anaconda3/envs/syft/lib/python3.6/site-packages/socketio/client.py", line 372, in _handle_event
    r = self._trigger_event(data[0], namespace, *data[1:])
  File "/home/marianne/anaconda3/envs

KeyboardInterrupt: 

In [16]:
fetched_plan.state

State: 

In [10]:
print(fetched_plan(sh_x).get())

KeyError: 'Object "42449684464" not found on worker!!!You just tried to interact with an object ID:42449684464 on <VirtualWorker id:me #objects:2> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [12]:
fetched_plan.id, fetched_plan.owner, sh_x.id

(41007873835, <VirtualWorker id:me #objects:2>, 26035843036)

In [8]:
fetched_plan.replace_worker_ids(b'bob', 'me')

In [9]:
fetched_plan.readable_plan

((26,
  (1,
   ((6,
     ((5, (b'torch.nn.functional.linear',)),
      None,
      (6,
       ((20, (43994367677, 26035843036, 'me', None, (10, (1, 1)), True)),
        (20, (75884231444, 42449684464, 'me', None, (10, (1, 1)), True)),
        (20, (66983378857, 9883980718, 'me', None, (10, (1,)), True)))),
      (0, ()))),
    (13184307804,)))),)

In [ ]:
fetched_plan.replace_ids(
            from_ids=fetched_plan.arg_ids, to_ids=fetched_plan.arg_ids, from_worker=self.id, to_worker=plan.id
        )
fetched_plan.replace_ids(
            from_ids=fetched_plan.result_ids, to_ids=fetched_plan.result_ids, from_worker=self.id, to_worker=plan.id
        )

In [6]:
sh_x.id

17055861697

In [3]:
data_shape = (1, 1)
data = torch.zeros(data_shape)
target = torch.tensor(1)

In [4]:
share_data = data.share(bob, alice, crypto_provider=crypto_provider)

In [5]:
me._objects[share_data.id] = share_data

In [8]:
new_model.share(bob, alice, crypto_provider=crypto_provider)

<Plan Net id:63682990815 owner:me built>

In [10]:
share_data.id

54374191711

In [9]:
new_model(share_data)

KeyError: 'Object "63251563507" not found on worker!!!You just tried to interact with an object ID:63251563507 on <VirtualWorker id:me #objects:2> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

We define the setting of the learning task

Second, the client has some data and would like to have predictions on it using the server's model. This client encrypts its data by sharing it additively across two workers `alice` and `bob`.
> SMPC uses crypto protocols which require to work on integers. We leverage here the PySyft tensor abstraction to convert PyTorch Float tensors into Fixed Precision Tensors using `.fix_precision()`. For example 0.123 with precision 2 does a rounding at the 2nd decimal digit so the number stored is the integer 12. 


Our model is now trained and ready to be provided as a service!

## Secure evaluation

Now, as the server, we send the model to the workers holding the data. Because the model is sensitive information (you've spent time optimizing it!), you don't want to disclose its weights so you secret share the model just like we did with the dataset earlier.

In [8]:
%%time
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

CPU times: user 1.98 s, sys: 784 ms, total: 2.77 s
Wall time: 2.53 s


<Net Net id:15703012123 owner:me built>

In [9]:
def test(model, data, target):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        shared_data = data.fix_precision().share(alice, bob, crypto_provider=crypto_provider)
        shared_target = target.fix_precision().share(alice, bob, crypto_provider=crypto_provider)
        output = model(shared_data)
        pred = output.argmax(dim=1)
        n_correct_priv += pred.eq(shared_target.view_as(pred)).sum()
        n_total += 1
# This 'test' function performs the encrypted evaluation. The model weights, the data inputs, the prediction and the target used for scoring are all encrypted!

# However as you can observe, the syntax is very similar to normal PyTorch testing! Nice!

# The only thing we decrypt from the server side is the final score at the end of our 200 items batches to verify predictions were on average good.      
        n_correct = n_correct_priv.copy().get().float_precision().long().item()

        print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
            n_correct, n_total,
            100. * n_correct / n_total))

        print('Prediction = {}, Label = {}'.format(pred.copy().get().float_precision(),
                                                   shared_target.copy().get().float_precision()))


In [10]:
%%time
test(model, data, target)

Test set: Accuracy: 0/1 (0%)
Prediction = tensor([0.]), Label = tensor([1.])
CPU times: user 1min 30s, sys: 34 s, total: 2min 4s
Wall time: 2min 1s


Et voilà! Here you are, you have learned how to do end to end secure predictions: the weights of the server's model have not leaked to the client and the server has no information about the data input nor the classification output!

Regarding performance, classifying one image takes **less than 0.1 second**, approximately **33ms** on my laptop (2,7 GHz Intel Core i7, 16GB RAM). However, this is using very fast communication (all the workers are on my local machine). Performance will vary depending on how fast different workers can talk to each other.

## Conclusion

You have seen how easy it is to leverage PyTorch and PySyft to perform practical Secure Machine Learning and protect users data, without having to be a crypto expert!

More on this topic will come soon, including convolutional layers to properly benchmark PySyft performance with respect to other libraries, as well as private encrypted training of neural networks, which is needed when a organisation resorts to external sensitive data to train its own model. Stay tuned!

If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways! 

### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)